In [ ]:
import dualcodec
import torchaudio
import torch
from IPython.display import Audio

In [ ]:
DEVICE = "cuda:3"

In [ ]:
model_id = "12hz_v1"
path = "/home/vansh/dualcodec-exp/output_checkpoints/dualcodec_25hzv1_finetune/checkpoint/epoch-0022_step-0110600_loss-70.316910-dualcodec_25hzv1_finetune"
name = "model.safetensors"

In [ ]:
dualcodec_model = dualcodec.get_model(model_id, path, name=name)
dualcodec_inference = dualcodec.Inference(dualcodec_model=dualcodec_model, device=DEVICE)

In [ ]:
model_id_base = "12hz_v1"
dualcodec_model_base = dualcodec.get_model(model_id_base)
dualcodec_inference_base = dualcodec.Inference(dualcodec_model=dualcodec_model_base, device=DEVICE)

In [ ]:
audio_tara, sr = torchaudio.load("audio_samples/tara.wav")
audio_tara = torchaudio.functional.resample(audio_tara, sr, 24000)
audio_tara = audio_tara.reshape(1,1,-1)
Audio(audio_tara.squeeze(0), rate=24000)

In [ ]:
audio_tara = audio_tara.to(DEVICE)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio_tara, n_quantizers=3)
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
print("*")
Audio(out_audio.cpu().squeeze(0), rate=24000)

In [ ]:
audio_sam, sr = torchaudio.load("audio_samples/sam.wav")
audio_sam = torchaudio.functional.resample(audio_sam, sr, 24000)
audio_sam = audio_sam.reshape(1,1,-1)
Audio(audio_sam.squeeze(0), rate=24000)

In [ ]:
audio_sam = audio_sam.to(DEVICE)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference_base.encode(audio_sam, n_quantizers=8)
out_audio = dualcodec_inference_base.decode(semantic_codes, acoustic_codes)
print("*")
Audio(out_audio.cpu().squeeze(0), rate=24000)

### Check

In [ ]:
from datasets import load_dataset
import os

local_dir = "/mnt/disks/emilia/emilia_dataset/Emilia/EN"
tar_paths = [filename for filename in os.listdir(local_dir) if filename.endswith(".tar")]
language = "EN"
            
dataset = load_dataset(
    local_dir,
    data_files={language.lower(): tar_paths},
    split=language.lower(),
    num_proc=40,
    cache_dir="/mnt/disks/emilia/emilia_cache",
)

In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "vanshjjw/amu-pushed-luna-4500r",
    split="train",
)       

In [ ]:
samples = []
for example in ds:
    if "audio" not in example or example["audio"] is None:
        continue
    audio_dict = example["audio"]
    waveform = torch.tensor(audio_dict["array"], dtype=torch.float32).unsqueeze(0)
    sr = int(audio_dict["sampling_rate"])
    
    if sr != 24000:
        waveform = torchaudio.functional.resample(waveform, sr, 24000)
    
    samples.append(
        {
            "mp3": {
                "array": waveform.numpy(),
                "sampling_rate": 24000,
            },
        }
    )

In [ ]:
samples[5]

In [ ]:
index = 5
audio_luna, sr = samples[index]['mp3']['array'], samples[index]['mp3']['sampling_rate']
audio_luna = torchaudio.functional.resample(audio_luna, sr, 24000)
audio_luna = audio_luna.reshape(1,1,-1)
Audio(audio_luna.squeeze(0), rate=24000)

In [ ]:
audio_luna = torch.tensor(audio_luna).to(DEVICE)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio_luna, n_quantizers=3)
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
print("*")
Audio(out_audio.cpu().squeeze(0), rate=24000)

In [ ]:
semantic_codes_base, acoustic_codes_base = dualcodec_inference_base.encode(audio_luna, n_quantizers=2)
out_audio_base = dualcodec_inference_base.decode(semantic_codes_base, acoustic_codes_base)
Audio(out_audio_base.cpu().squeeze(0), rate=24000)